# Subsections Issue

## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Function to compare context for two sections.

In [3]:
def compare_context(section1, section2):
    ctx_template = '{key:16s}:\t{item1:16s}\t{item2:16s}'
    context_1 = section1.context
    context_2 = section2.context
    keys_1 = set(context_1.keys())
    keys_2 = set(context_2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = context_1.get(key, '')
        item2 = context_2.get(key, '')
        ctx_str = ctx_template.format(key=str(key), item1=str(item1), item2=str(item2))
        print(ctx_str)

# Simple sections experimenting with start and end settings

### Defining a Top Section that calls two single line subsections.

#### Two Subsection definitions 

In [27]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Including unwanted text in between the start and end of subsection C

In [29]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]



[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


#### Setting `keep_partial=True`

In [80]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

[[['StartSection Name: A'], ['EndSection Name: A']],
 [['StartSection Name: B'], []]]


![Bad](../examples/error.png) Expecting `['StartSection C']`, 
but not the following empty list.
- That may not be a bad thing.
- Just need to understand why it happens

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>
          ]
        </code></td></tr>
</table>

#### Dropping the corresponding *EndSection* for a *StartSection*.

In [34]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  # Missing 'EndSection B',
    
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]

top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT2))

![Bad](../examples/error.png) Expecting `['StartSection B']`, 
but not the following empty list.
- 3rd section group never starts because **ignored** *Top Section* break line 
  occurs before next **EndSection**, so 2nd never finishes.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td></tr>
</table>

**********

# Section Break options

## Three line sections

In [44]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [46]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,
    subsections=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['StartSection A',
  'MiddleSection A',
  'EndSection A',
  'Unwanted text between sections',
  'StartSection B',
  'MiddleSection B',
  'EndSection B',
  'StartSection C',
  'MiddleSection C',
  'EndSection C',
  'Even more text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
</table>



### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )
```

In [47]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,    
    subsections=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

[['StartSection A', 'MiddleSection A', 'EndSection A']]
[['StartSection B', 'MiddleSection B', 'EndSection B']]
[['StartSection C', 'MiddleSection C', 'EndSection C']]
[]


- ![Good](../examples/Valid.png) Includes all three lines of first section in single sub-list.
- skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
</tr>
<tr><td><code>[]</code></td><td><code>[]</code></td></tr>
</table>



### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section 
    )
```

In [48]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png)Includes all three lines of each section in its own sub-list.
- Skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **Before** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section 
    )
```

In [49]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A',
   'MiddleSection A',
   'EndSection A',
   'Unwanted text between sections']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C',
   'MiddleSection C',
   'EndSection C',
   'Even more text to be ignored']]]


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- Includes *'Unwanted text between sections'* because the end of section A is 
  triggered by *'StartSection B'*. 
- Includes *'Even more text to be ignored'* because there are no more 
  *'StartSection'* lines to trigger an `end_section` break.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  </tr>
</table>

### Add `end_on_first_item=True` to Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section 
    )
```

In [50]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

test_iter = iter(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

[]
[]
[]
[]


- ![Good](../examples/Valid.png) Results in empty list because it both starts 
  and ends on first *'StartSection'*.

|Expected|Actual|
|-|-|
|[]|[]|

### <span style="color: red; font-weight: Bold">Endless loop with `multi_section`</span>

<span style="color: red; font-weight: Bold; font-variant: small-caps">
Do Not Try This
</span>

- Use `multi_section` to read multiple empty sections
```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section 
    )
multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

- `full_section` both starts and ends on first *'StartSection'*.
- No progress is made in in iterating through the source.
- Trying to read multiple sections will result in an endless loop.

**********

## SubSection Break Options
- Section break checks start with the top most section
- The section level containing the break will affect the way the list are nested.

### Add `start_section` to *sub_section* and remove it from *full_section*

- SubSection start **Before** *StartSection*
- Section end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [51]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

In [52]:
pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) behaves the same way as when `start_section` 
 was defined in *full_section*.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Add `end_section` to *sub_section* and remove it from *full_section*

- SubSection end **After** *EndSection*
- Section start **Before** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before')
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [53]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A'],
  ['Unwanted text between sections',
   'StartSection B',
   'MiddleSection B',
   'EndSection B'],
  ['StartSection C', 'MiddleSection C', 'EndSection C'],
  ['Even more text to be ignored']]]


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- `sub_section` has no `start_section` defined, so the it starts immediately. 
- `full_section` never ends and so the second sub-section starts right after 
  the first one ends. 
- **Note:** 
    - sub-list nesting is different:
        > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
        > VS.<br>
        > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>
    - *'Even more text to be ignored'* is in its own sub-section.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  </tr>
</table>

### Add both `start_section` and `end_section` to *sub_section*. No breaks defined in *full_section*

- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [54]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A'],
  ['StartSection B', 'MiddleSection B', 'EndSection B'],
  ['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) similar results to having both breaks defined in *full_section*.
- **Note:** sub-list nesting is different:
    > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
    > VS.<br>
    > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  </tr>
</table>

### Read from *full_section* instead of *multi_section*

In [55]:
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['StartSection A', 'MiddleSection A', 'EndSection A'],
 ['StartSection B', 'MiddleSection B', 'EndSection B'],
 ['StartSection C', 'MiddleSection C', 'EndSection C']]


- ![Good](../examples/Valid.png) Similar results to reading *multi_section*.

- **Note:** One less list level.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  </tr>
</table>

### Add `start_section` and `end_section` to both *sub_section* and *full_section*.

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [56]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) behaves the same way as when *sub_section* 
  did not have any breaks defined (including list levels).

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

### Change Start Sub-Section to **After**

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [57]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[[['MiddleSection A', 'EndSection A']],
 [['MiddleSection B', 'EndSection B']],
 [['MiddleSection C', 'EndSection C']]]


- ![Good](../examples/Valid.png) *StartSection* lines are dropped.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

**********

# DONE TO HERE

### Run with two-line sections:
> `[`<br>
> `'Text to be ignored',`<br>
> `'StartSection Name: A',`<br>
> `'EndSection Name: A',`<br>
> `'Text between sections',`<br>
> `'StartSection Name: B',`<br>
> `'EndSection Name: B',`<br>
> `'More text to be ignored'`<br>
> `]`<br>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [58]:
pprint(multi_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/error.png) Results in blank sub-list.
<table><thead><th>Expected</th><th>Actual</th></thead>
  <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
    ]</code></td>
  <td><code>[]</code></td></tr>
</table>

In [59]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

[]


In [60]:
GENERIC_TEST_TEXT

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']

In [61]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/error.png) Results in blank sub-list.
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [62]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [63]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [64]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition and set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [65]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [66]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A'], ['EndSection Name: B'], []]
  </code></td></tr>
</table>

### Add *Start After StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [67]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A']]
  </code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [68]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to *Before EndSection*, *End* to *`True` (Always Break)* and *Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [69]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [70]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section

- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td>
<td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __After__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [71]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in one line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

## Three line sections

In [72]:
GENERIC_TEST_TEXT3 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'Random text in the middle of a section',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

#### Single line subsection definitions 

In [73]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

middle_sub_section = Section(
    section_name='MiddleSubSection',
    start_section=SectionBreak('MiddleSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***three_part_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

- Contains 3 subsections:
    > `[start_sub_section, middle_sub_section, end_sub_section]`

In [74]:
three_part_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, middle_sub_section, end_sub_section]
    )
pprint(three_part_section.read(GENERIC_TEST_TEXT3))

[[['StartSection A'], ['MiddleSection A'], ['EndSection A']],
 [['StartSection B'], ['MiddleSection B'], ['EndSection B']],
 [['StartSection C'], ['MiddleSection C'], ['EndSection C']]]


![Good](../examples/Valid.png) All 3 sections and subsections are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td></tr>
</table>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [75]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT3))

[['StartSection A', 'MiddleSection A', 'EndSection A']]


In [76]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section]
    )
pprint(top_section.read(GENERIC_TEST_TEXT3))

[[['StartSection A'], ['EndSection A']],
 [['StartSection B'], ['EndSection B']],
 [['StartSection C'], ['EndSection C']]]


In [77]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='After')
    )

In [78]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=True
    )

In [79]:
pprint(list(test_iter.previous_items))

AttributeError: 'list_iterator' object has no attribute 'previous_items'

In [ ]:
pprint(list(test_iter.future_items))

['Even more text to be ignored']


In [ ]:
compare_context(full_section, sub_section)

Status          :	Not Started     	Not Started     
Current Section :	Full            	SubSection      
